In [4]:
import numpy as np
import pandas as pd
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence, plot_objective
import matplotlib.pyplot as plt
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
from models.standard_dc import TeamModel
from data.fetch_match_data import load_data


In [5]:
def rolling_window_evaluation(matches, shot_data, 
                             start_date='2024-10-01',  # Start evaluation from this date
                             end_date=None,            # Optional end date
                             window_size=365, 
                             n_simulations=25, 
                             epsilon=0.01, 
                             season_penalty=0.75,
                             standard_model=False):
    # Convert to DataFrame if not already
    if not isinstance(matches, pd.DataFrame):
        matches_df = pd.DataFrame(matches)
    else:
        matches_df = matches.copy()
    
    # Ensure dates are datetime objects
    matches_df['match_date'] = pd.to_datetime(matches_df['match_date'])
    
    # Sort by date
    matches_df = matches_df.sort_values('match_date')
    
    # Filter matches for evaluation (only those after start_date)
    eval_start = pd.to_datetime(start_date)
    if end_date:
        eval_end = pd.to_datetime(end_date)
        eval_matches = matches_df[(matches_df['match_date'] >= eval_start) & 
                                 (matches_df['match_date'] <= eval_end)]
    else:
        eval_matches = matches_df[matches_df['match_date'] >= eval_start]
    
    # Group evaluation matches by week
    eval_matches['prediction_week'] = eval_matches['match_date'].dt.to_period('W')
    
    # Get unique weeks for evaluation
    prediction_weeks = eval_matches['prediction_week'].unique()
    
    # Store results
    results = []
    detailed_predictions = []
    
    # For each prediction window
    for week in prediction_weeks:
        print(f"\nEvaluating prediction week: {week}")
        
        # Define cutoff dates
        prediction_start = week.start_time
        prediction_end = week.end_time
        training_end = prediction_start - pd.Timedelta(days=1)
        training_start = training_end - pd.Timedelta(days=window_size)
        
        # Get training data (from the entire dataset, not just eval period)
        training_data = matches_df[(matches_df['match_date'] >= training_start) & 
                                  (matches_df['match_date'] <= training_end)]
        
        # Get prediction data for this week
        prediction_data = matches_df[(matches_df['match_date'] >= prediction_start) & 
                                    (matches_df['match_date'] <= prediction_end)]
        
        # Skip if not enough data
        if len(training_data) < 10 or len(prediction_data) < 1:
            print(f"  Skipping week {week} due to insufficient data")
            continue
        
        print(f"  Training on {len(training_data)} matches from {training_start.date()} to {training_end.date()}")
        print(f"  Predicting {len(prediction_data)} matches from {prediction_start.date()} to {prediction_end.date()}")
        
        # Convert training data to list of dictionaries
        training_matches = training_data.to_dict('records')
        
        # Create model and fit on training data
        if standard_model:
            model = TeamModel()
            model.fit_models(training_matches, epsilon=epsilon, season_penalty=season_penalty, days_ago=900)
        else:
            model = TeamModel(n_simulations=n_simulations)
            model.fit_models(training_matches, shot_data, epsilon=epsilon, season_penalty=season_penalty)
        
        # Make predictions for each match in the prediction window
        week_preds = []
        week_actuals = []
        match_details = []
        
        for _, match in prediction_data.iterrows():
            home_team = match['home_team']
            away_team = match['away_team']
            
            # Skip if teams not in model
            if home_team not in model.team_attack or away_team not in model.team_attack:
                continue
            
            # Predict
            prediction = model.predict_match(home_team, away_team)
            
            # Store prediction and actual
            pred_home = prediction['home_goals']
            pred_away = prediction['away_goals']
            actual_home = match['home_goals']
            actual_away = match['away_goals']
            
            week_preds.append((pred_home, pred_away))
            week_actuals.append((actual_home, actual_away))
            
            # Store detailed match prediction
            match_details.append({
                'week': str(week),
                'match_date': match['match_date'],
                'home_team': home_team,
                'away_team': away_team,
                'predicted_home_goals': pred_home,
                'predicted_away_goals': pred_away,
                'actual_home_goals': actual_home,
                'actual_away_goals': actual_away,
                'home_error': abs(pred_home - actual_home),
                'away_error': abs(pred_away - actual_away),
                'total_error': abs(pred_home - actual_home) + abs(pred_away - actual_away),
                'epsilon': epsilon,
                'season_penalty': season_penalty,
                'n_simulations': n_simulations
            })
        
        # Calculate metrics
        if len(week_preds) > 2:
            home_mae = np.mean([abs(pred[0] - actual[0]) for pred, actual in zip(week_preds, week_actuals)])
            away_mae = np.mean([abs(pred[1] - actual[1]) for pred, actual in zip(week_preds, week_actuals)])
            total_mae = np.mean([abs(pred[0] - actual[0]) + abs(pred[1] - actual[1]) for pred, actual in zip(week_preds, week_actuals)])
            
            print(f"  Results - Home MAE: {home_mae:.3f}, Away MAE: {away_mae:.3f}, Total MAE: {total_mae:.3f}")
            
            # Store results
            results.append({
                'week': str(week),
                'week_start': prediction_start,
                'week_end': prediction_end,
                'num_matches': len(week_preds),
                'home_mae': home_mae,
                'away_mae': away_mae,
                'total_mae': total_mae,
                'epsilon': epsilon,
                'season_penalty': season_penalty,
                'n_simulations': n_simulations
            })
            
            # Add match details
            detailed_predictions.extend(match_details)
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    detailed_df = pd.DataFrame(detailed_predictions)
    
    # Calculate aggregate metrics
    if len(results_df) > 0:
        avg_home_mae = results_df['home_mae'].mean()
        avg_away_mae = results_df['away_mae'].mean()
        avg_total_mae = results_df['total_mae'].mean()
        
        print(f"\nOverall Results:")
        print(f"Average Home MAE: {avg_home_mae:.3f}")
        print(f"Average Away MAE: {avg_away_mae:.3f}")
        print(f"Average Total MAE: {avg_total_mae:.3f}")
    else:
        print("\nNo valid predictions in the evaluation period.")
    
    return results_df, detailed_df

In [6]:
# Load your data
shot_data, match_stats = load_data()

# Define parameter combinations to test
params_to_test = [
    {'epsilon': 0.00005, 'season_penalty': 1, 'n_simulations': 10},
    {'epsilon': 0.0001, 'season_penalty': 1, 'n_simulations': 10},
    {'epsilon': 0.0002, 'season_penalty': 1, 'n_simulations': 10},
    {'epsilon': 0.001, 'season_penalty': 1, 'n_simulations': 10}
]

# Store results for each parameter combination
all_parameter_results = []

# Test each parameter combination
for params in params_to_test:
    print(f"\n\nTesting parameters: {params}")
    epsilon = params['epsilon']
    season_penalty = params['season_penalty']
    n_simulations = params['n_simulations']
    
    # Run evaluation
    results_df, detailed_df = rolling_window_evaluation(
        match_stats,
        shot_data,
        start_date='2024-10-01',
        window_size=365,
        n_simulations=n_simulations,
        epsilon=epsilon,
        season_penalty=season_penalty,
        standard_model=True
    )
    
    # Store overall metrics for this parameter combination
    if len(results_df) > 0:
        param_result = {
            'epsilon': epsilon,
            'season_penalty': season_penalty,
            'n_simulations': n_simulations,
            'home_mae': results_df['home_mae'].mean(),
            'away_mae': results_df['away_mae'].mean(),
            'total_mae': results_df['total_mae'].mean(),
            'num_matches': results_df['num_matches'].sum()
        }
        all_parameter_results.append(param_result)
        
        # Save detailed results to CSV
        filename = f"results_eps{epsilon}_sp{season_penalty}_sim{n_simulations}.csv"
        detailed_df.to_csv(filename, index=False)
        print(f"Saved detailed results to {filename}")

# Create summary DataFrame and find best parameters
if all_parameter_results:
    summary_df = pd.DataFrame(all_parameter_results)
    summary_df = summary_df.sort_values('total_mae')
    
    print("\n===== Parameter Results (sorted by total MAE) =====")
    print(summary_df)
    
    best_params = summary_df.iloc[0]
    print("\n===== Best Parameter Combination =====")
    print(f"Epsilon: {best_params['epsilon']}")
    print(f"Season Penalty: {best_params['season_penalty']}")
    print(f"Simulations: {int(best_params['n_simulations'])}")
    print(f"Home MAE: {best_params['home_mae']:.3f}")
    print(f"Away MAE: {best_params['away_mae']:.3f}")
    print(f"Total MAE: {best_params['total_mae']:.3f}")
    
    # Save summary
    summary_df.to_csv("parameter_summary.csv", index=False)
else:
    print("No valid results were produced.")



Testing parameters: {'epsilon': 5e-05, 'season_penalty': 1, 'n_simulations': 10}

Evaluating prediction week: 2024-09-30/2024-10-06
  Training on 380 matches from 2023-09-30 to 2024-09-29
  Predicting 11 matches from 2024-09-30 to 2024-10-06
Optimizing for 380 matches with 23 teams
Match 0: {'match_url': 'https://fbref.com/en/matches/ec4145b4/Tottenham-Hotspur-Liverpool-September-30-2023-Premier-League', 'match_date': Timestamp('2023-09-30 00:00:00'), 'home_team': 'Tottenham', 'away_team': 'Liverpool', 'season': 2023, 'home_goals': 1, 'home_xg': 2.2399999999999998, 'home_psxg': 2.22, 'away_goals': 1, 'away_xg': 1.33, 'away_psxg': 0.33, 'days_from_ref': 365}
Match 1: {'match_url': 'https://fbref.com/en/matches/2df9a3a1/Aston-Villa-Brighton-and-Hove-Albion-September-30-2023-Premier-League', 'match_date': Timestamp('2023-09-30 00:00:00'), 'home_team': 'Aston Villa', 'away_team': 'Brighton', 'season': 2023, 'home_goals': 5, 'home_xg': 1.62, 'home_psxg': 3.25, 'away_goals': 1, 'away_xg': 

C:\Users\Owner\AppData\Local\Temp\ipykernel_26988\4291364502.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_matches['prediction_week'] = eval_matches['match_date'].dt.to_period('W')


KeyboardInterrupt: 